In [11]:
import pandas as pd
from libraries.db import DatabaseProxy

In [12]:
df = pd.read_csv("input/50qa-pairs.csv", encoding="latin-1", dtype="str")
df.head()

,question,answer,partners_personnel_selected
0,How many candidates do you have in your database?,"As of today, 4/25/2023, we have the following:...",0
1,How often do you review and refresh your datab...,Our sourcing platform is updated throughout th...,0
2,If you are a member of the American Staffing A...,Partners Personnel is not currently a member o...,0
3,How do you determine which employees need cove...,ACA compliance states that anyone who on avera...,0
4,Can you commit to meet emergency requests and/...,"Yes, Depends on each site. Minimum lead times ...",0


In [13]:
import re


non_alphanumeric = set()
pattern = r"[^a-zA-Z0-9\s]"

# get all non-alphanumeric characters in the questions and answers
for index, row in df.iterrows():
    question = row["question"]
    answer = str(row["answer"])
    non_alphanumeric.update(set(re.findall(pattern, question)))
    non_alphanumeric.update(set(re.findall(pattern, answer)))

In [14]:
non_alphanumeric

{'%', '&', '(', ')', ',', '-', '.', '/', ':', '>', '?', '½', '¿', 'ï'}

In [15]:
mapping = {"¥": "-", "É": "...", "Ð": "—", "Ñ": "—", "Ò": '"', "Ó": '"', "Õ": "'"}


def replace_non_alphanumeric(text):
    text = str(text)
    for key, value in mapping.items():
        text = text.replace(key, value)
    return text


df["question"] = df["question"].apply(replace_non_alphanumeric)
df["answer"] = df["answer"].apply(replace_non_alphanumeric)

In [16]:
db = DatabaseProxy(reset=True)

In [17]:
from libraries.db.model import Answer, Question


for _, row in df.iterrows():
    answer = Answer(text=str(row["answer"]).strip())
    answer = db.insert_answer(answer)
    question = Question(text=str(row["question"]).strip(), answer_id=answer.id)
    db.insert_question(question)

## View answers

In [18]:
db.get_answers()[:5]

[Answer(modified_datetime=datetime.datetime(2023, 11, 17, 22, 26, 43), id=1, owner_organization=None, text='As of today, 4/25/2023, we have the following:\n2060 associates who have successfully completed assignments and are available for work\n3464 associate who have been onboarded within the last 30 days and are available for work\n4117 associates going through the application process\nIn total we have 57083 associates active and unassigned in our system.', created_datetime=datetime.datetime(2023, 11, 17, 22, 26, 43)),
 Answer(modified_datetime=datetime.datetime(2023, 11, 17, 22, 26, 43), id=2, owner_organization=None, text='Our sourcing platform is updated throughout the day and allows our teams to view associate who have recently completed assignments successfully, those who have been newly onboarded and those in the application process. This allows our teams to view the most up to date information with associates who are actively seeking employment.', created_datetime=datetime.date

In [19]:
db.get_answer_by_id(152)

In [20]:
len(db.get_questions())

50